In [ ]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)

In [ ]:
object <- readRDS("./RDS/250520.combined_3.4_visium_label_update_hires.rds")

In [ ]:
object

In [ ]:
object <- subset(object, idents = c("Neutrophilic pustule", "Eccrine gland coil", "Adipocyte", "Eccrine gland duct"), invert = TRUE)


In [ ]:
ref <- readRDS("/data/User/revolvefire/projects/3.PPP_231118/231128_PPP/RDS/251115_integrated_total_rawcounts.rds")

In [ ]:
ref <- NormalizeData(ref)

In [ ]:
ref <- FindVariableFeatures(ref)

In [ ]:
Idents(ref) <- "label2"

In [ ]:
DimPlot(ref, group.by = "label2", label = TRUE)

In [ ]:
ref <- subset(ref, idents = c("Melanocyte"), invert = TRUE)

In [ ]:
DimPlot(ref, group.by = "label3", label = TRUE)

In [ ]:
ref@meta.data$label3 <- as.character(ref@meta.data$label3)
ref@meta.data$label3[ref@meta.data$label3 %in% c("BK-1", "BK-2", "BK-3")] <- "BK-1-3"
ref@meta.data$label3[ref@meta.data$label3 %in% c("SBK-1", "SBK-2")] <- "SBK-1-2"
ref@meta.data$label3[ref@meta.data$label3 %in% c("SBK-3", "SBK-4")] <- "SBK-3-4"

In [ ]:
ref@meta.data$label3[ref@meta.data$label3 %in% c("Superficial FIB", "FRC-like FIB", "Inf universal FIB", "Universal FIB")] <- "Fibroblast"

In [ ]:
anchors <- FindTransferAnchors(reference = ref, query = object, normalization.method="LogNormalize")
predictions.assay <- TransferData(anchorset = anchors, refdata = ref$label3, prediction.assay = TRUE,
    weight.reduction = object[["pca"]], dims = 1:30)
object[["predictions"]] <- predictions.assay

In [ ]:
# Subset for slide1
s1 <- subset(object, subset = slide == "slide1")

# Subset for slide2
s2 <- subset(object, subset = slide == "slide2")

In [ ]:
library(RColorBrewer)

In [ ]:
DefaultAssay(s1) <- "predictions"
immune_labels <- c("Th17", "CD4em", "CD4cm", "Treg", "CD8/NK", "pDC", "cDC2-1", "cDC2-2", "mDC",  "Fibroblast")
niche_cells <- WhichCells(s1, expression = label == "Lymphoid-CCL19+ niche")
scores <- FetchData(s1, vars = immune_labels, layer = "data")[niche_cells, ]
thresholds <- apply(scores, 2, quantile, probs = 0.8, na.rm = TRUE)

for (ct in immune_labels) {
  scores[[ct]][scores[[ct]] < thresholds[ct]] <- NA
}

max_labels <- apply(scores, 1, function(x) {
  if (all(is.na(x))) return(NA)
  colnames(scores)[which.max(x)]
})
max_scores <- apply(scores, 1, function(x) ifelse(all(is.na(x)), NA, max(x, na.rm = TRUE)))


s1$top_label <- NA
s1$score <- NA
s1$top_label[niche_cells] <- max_labels
s1$score[niche_cells] <- max_scores

s1$top_label <- factor(s1$top_label, levels = immune_labels)

palette <- c(
  "Th17"       = "#EF476F",
  "CD4em"      = "#fb8500",
  "CD4cm"       = "#ffd60a",
    "Treg" = "#eaff99",
  "CD8/NK"     = "#91cb3e",
  "pDC"        = "#20bf55",
  "cDC2-1"     = "#00b4d8",
  "cDC2-2"     = "#0077b6",
  "mDC"        = "#8f03ff",
  "Fibroblast" = "#a4b0b4"
)

In [ ]:
p <- SpatialPlot(
  object = subset(s1, cells = niche_cells),
  group.by = "top_label",
  pt.size.factor = 2.5,
  image.scale = "hires",
  image.alpha = 0.2,
  cols = palette
) +
 
  theme(
    legend.text = element_text(size = 12),
    legend.title = element_text(size = 14),
    legend.key.size = unit(1.5, "lines")
  ) +
  guides(
    fill = guide_legend(override.aes = list(size = 6))  
  )


print(p)

In [ ]:
rotate_image <- function(p, rot_angle) {
  gt <- ggplot_gtable(ggplot_build(p))
  panel_idx <- which(gt$layout$name == "panel")
  rot_vp <- viewport(angle = rot_angle)
  gt[["grobs"]][[panel_idx]] <- editGrob(gt[["grobs"]][[panel_idx]], vp = rot_vp)
  p_rot <- ggdraw() + draw_grob(gt)
  
  return(p_rot)
}

In [ ]:
library(cowplot)
library(ggplot2)
library(grid)

In [ ]:
p_rot <- rotate_image(p, 270)

In [ ]:
p_rot

In [ ]:
ggsave("./revision_figure/251115_s1_lymphoid_subcluster4.jpg", plot = p_rot, device = "jpg", dpi=1200, width = 6, height = 10)

In [ ]:
DefaultAssay(s2) <- "predictions"

immune_labels <- c("Th17", "CD4em", "CD4cm", "Treg", "CD8/NK", "pDC",   "cDC2-1", "cDC2-2","mDC", "Fibroblast")
niche_cells_s2 <- WhichCells(s2, expression = label == "Lymphoid-CCL19+ niche")
scores <- FetchData(s2, vars = immune_labels, layer = "data")[niche_cells_s2, ]
thresholds <- apply(scores, 2, quantile, probs = 0.8, na.rm = TRUE)

for (ct in immune_labels) {
  scores[[ct]][scores[[ct]] < thresholds[ct]] <- NA
}

max_labels <- apply(scores, 1, function(x) {
  if (all(is.na(x))) return(NA)
  colnames(scores)[which.max(x)]
})
max_scores <- apply(scores, 1, function(x) ifelse(all(is.na(x)), NA, max(x, na.rm = TRUE)))

s2$top_label <- NA
s2$score <- NA
s2$top_label[niche_cells_s2] <- max_labels
s2$score[niche_cells_s2] <- max_scores

s2$top_label <- factor(s2$top_label, levels = immune_labels)

palette <- c(
  "Th17"       = "#EF476F",
  "CD4em"      = "#fb8500",
  "CD4cm"       = "#ffd60a",
    "Treg" = "#eaff99",
  "CD8/NK"     = "#91cb3e",
  "pDC"        = "#20bf55",
  "cDC2-1"     = "#00b4d8",
  "cDC2-2"     = "#0077b6",
  "mDC"        = "#8f03ff",
  "Fibroblast" = "#a4b0b4"
)

In [ ]:
p <- SpatialPlot(
  object = subset(s2, cells = niche_cells_s2),
  group.by = "top_label",
  pt.size.factor = 2.5,
  image.scale = "hires",
  image.alpha = 0.2,
  cols = palette
) +
  theme(
    legend.text = element_text(size = 12),
    legend.title = element_text(size = 14),
    legend.key.size = unit(1.5, "lines")
  ) +
  guides(
    fill = guide_legend(override.aes = list(size = 6))  
  )

print(p)

In [ ]:
rotate_image <- function(p, rot_angle) {
  gt <- ggplot_gtable(ggplot_build(p))
  panel_idx <- which(gt$layout$name == "panel")
  rot_vp <- viewport(angle = rot_angle)
  gt[["grobs"]][[panel_idx]] <- editGrob(gt[["grobs"]][[panel_idx]], vp = rot_vp)
  p_rot <- ggdraw() + draw_grob(gt)
  
  return(p_rot)
}

In [ ]:
p_rot <- rotate_image(p, 270)

In [ ]:
p_rot

In [ ]:
ggsave("./revision_figure/251115_s2_lymphoid_subcluster4.jpg", plot = p_rot, device = "jpg", dpi=1200, width = 6, height = 10)

In [ ]:
sum(is.na(s2$top_label[niche_cells_s2]))  

In [ ]:
sum(is.na(s1$top_label[niche_cells]))  